In [ ]:
import os
import json
import re
import spacy
import pandas as pd
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()
import re
import pickle

In [ ]:
# Define Path
path=r'/opt/ml-training/Notebooks/Sahil/KnowledgeGraph/DataSet/Case_Study'

In [ ]:
# Get Entities
def get_entities_update(sent):
    
    ent1 = ""
    ent2 = ""
    
    
    prv_tok_dep = ""    # dependency tag of previous token in the sentence
    prv_tok_text = ""   # previous token in the sentence
    
    prefix = ""
    modifier = ""
    
    list_ = []
    
    for tok in nlp(sent):
        if tok.dep_ != "punct":
            if tok.dep_ == "compound":
                prefix = tok.text
                if prv_tok_dep == "compound":
                    prefix = prv_tok_text + " "+ tok.text
    
            if tok.dep_.endswith("mod") == True:
                modifier = tok.text
                if prv_tok_dep == "compound":
                    modifier = prv_tok_text + " "+ tok.text
    
            if tok.dep_.find("subj") == True:
                #ent1 = modifier +" "+ prefix + " "+ tok.text
                ent = ''
                ent = modifier +" "+ prefix + " "+ tok.text
                prefix = ""
                modifier = ""
                prv_tok_dep = ""
                prv_tok_text = ""
                list_.append(ent)
    
            if tok.dep_.find("obj") == True:
                ent = ''
                ent = modifier +" "+ prefix +" "+ tok.text
                list_.append(ent)
    
            prv_tok_dep = tok.dep_
            prv_tok_text = tok.text
    #return [ent1.strip(), ent2.strip()]
    return list_

def RemoveStopWords(List_):
    #Single_Word_Tokens = []
    List_without_space = [x.strip() for x in List_]
    for key in List_without_space:
        Token = ''.join(key).strip()
        Number_of_words = len(Token.split(" "))
    
    #print(key)
        if Number_of_words== 1:
            #Single_Word_Tokens.append(Token)
            doc = nlp(Token) # Convert to Spacy Object
            for token in doc:
                if token.is_stop:
                    token =str(token) # Convert from Spacy to String for list operations
                    #print(token)
                    try:
                        List_without_space.remove(token)
                    except:
                        pass
                        
                
    return List_without_space 

In [ ]:
List_of_Values = []
counter = 0
for i in os.walk(path):
    #print(i)
    for j in i[2]:
        
        StringVal = ''
        List_of_Sentences = []
        file = open(path+'/'+j, encoding="utf8")
        #print(file)
        data=json.loads(file.read())
        StringVal = data['body_t']
        #List_of_Values.append(get_entities_Noun(StringVal))
        List_of_Sentences = StringVal.split("\n")
        List_of_Sentences = [re.sub('[^a-zA-Z0-9]+'," ",x) for x in List_of_Sentences ]
        counter  = counter + 1
        print(counter)
        for each in List_of_Sentences:
        #SearchString = str(each) #  = ' 49 EDGE Project Mgmt Work Mgmt INREV Initiate Project Reviews PM10 Project Mgmt Work Mgmt '
            if len(each) >1:
                List_of_Values.append(get_entities_update(each))
            else:
                pass

In [ ]:
print("List of Values Length:",len(List_of_Values))
#Flatten the List
List_consolidated = [val for each in List_of_Values for val in each]
print("List_consolidated Length:",len(List_consolidated))
# Save Original File
with open('CaseStudy.pkl', 'wb') as f:
    pickle.dump(List_consolidated, f)

In [ ]:
word_freq = Counter(List_consolidated)
common_words = word_freq.most_common(1000)
print (common_words)

In [ ]:
# Remove Stopwords
ConsoladatedList= []
ConsoladatedList = RemoveStopWords(List_consolidated)
len(ConsoladatedList)

In [ ]:
# Save StopWords File
with open('CaseStudy_SW.pkl', 'wb') as f:
    pickle.dump(ConsoladatedList, f)

In [ ]:
with open('CaseStudy_SW.pkl', 'rb') as f:
    ConsoladatedList = pickle.load(f)
len(ConsoladatedList)

In [ ]:
# Remove Single Characters tokens
[ConsoladatedList.remove(each) for each in ConsoladatedList if len(each.strip()) ==1]
len(ConsoladatedList)

In [ ]:
ConsoladatedList = [re.sub('\S+@\S+'," ",x) for x in ConsoladatedList ] # Replace EmailAddress
ConsoladatedList = [re.sub("e.g. "," ",x) for x in ConsoladatedList]
len(ConsoladatedList)

In [ ]:
List_Of_Removals = [' ', '', ' ', 'Confidential Proprietary', 'Proprietary', 'Confidential', 'Confidential Proprietary', 
                    'Don t', 'Status e','g Release', 'N A']
[ConsoladatedList.remove(each) for each in ConsoladatedList if each in List_Of_Removals]
len(ConsoladatedList)

In [ ]:
for x in list(ConsoladatedList):
    token = x.split(" ")
    if len(token) == 2:
        if token[0] == token[1]:
            ConsoladatedList.remove(x)

len(ConsoladatedList)

In [ ]:
word_freq = Counter(ConsoladatedList)
common_words = word_freq.most_common(1050)
print (common_words)

In [ ]:
Two_Tokens_Removals = ['you','now', 'we', 'it', 'that', 'so', 'what', 'this', 'they','us', 'more', 'however', 'when', 'where', 
                    'then', 'them', 'just', 'll', 'here', 'other', 'which', 'why', 'actually', 'other', 'their', 'how',
                     'really', 'there', 'again']

for each in list(ConsoladatedList):
    token = each.split(" ")
    if token[0].lower() in Two_Tokens_Removals:
        ConsoladatedList.remove(each)

len(ConsoladatedList)

In [ ]:
word_freq = Counter(ConsoladatedList)
common_words = word_freq.most_common(1050)
print (common_words)

In [ ]:
List_Of_Removals = ['N ERROR', 'me', 'use', 'way', '0 N ERROR', 'ERROR N ERROR', 'lot', 't i', 'example', 'one  I']
[ConsoladatedList.remove(each) for each in ConsoladatedList if each in List_Of_Removals]
len(ConsoladatedList)

In [ ]:
word_freq = Counter(ConsoladatedList)
common_words = word_freq.most_common(1050)
print (common_words)

In [ ]:
## Capture Tags with Length greater than 1
Len_Greater1 = []
for each in list(ConsoladatedList):
    token = each.split(" ")
    if len(token) >1:
        Len_Greater1.append(each)
len(Len_Greater1)

In [ ]:
word_freq = Counter(Len_Greater1)
common_words = word_freq.most_common(1050)
print (common_words)

In [ ]:
for x in Len_Greater1:
    token = x.split(" ")
    if len(token) == 2:
        if token[0] == token[1]:
            Len_Greater1.remove(x)

len(Len_Greater1)

In [ ]:
Two_Tokens_Removals = ['you','now', 'we', 'it', 'that', 'so', 'what', 'this', 'they','us', 'more', 'however', 'when', 'where', 
                    'then', 'them', 'just', 'll', 'here', 'other', 'which', 'why', 'actually', 'other', 'their', 'how',
                     'really', 'there', 'again']

for each in list(Len_Greater1):
    token = each.split(" ")
    if token[0].lower() in Two_Tokens_Removals:
        Len_Greater1.remove(each)

len(Len_Greater1)

In [ ]:
# Save to CSV

List_Phrase = [phrase for phrase,frequency in common_words]
List_Frequency = [frequency for phrase,frequency in common_words]

print(len(List_Phrase))
print(len(List_Frequency))

#for k,v in common_words:
#    print(v)
#import pandas as pd

DF = pd.DataFrame()
DF['Phrases'] = List_Phrase
DF['Frequency'] = List_Frequency

DF.to_excel("Client_win_OnlyNoun.xlsx")